In [1]:
from nba_calculator.utils import CalculatorUtils, EnvConfig

In [2]:
# Configuration
ENDPOINT = EnvConfig.str("ENDPOINT", "http://127.0.0.1:8000/api/v1/internal/events/nba-calculation")
NBA_LIST_ENDPOINT = EnvConfig.str("NBA_LIST_ENDPOINT", "http://127.0.0.1:8000/api/v1/nba")
EMPLOYEE_THRESHOLD = EnvConfig.int("EMPLOYEE_THRESHOLD", 3)
NBA_DEFINITION_ID = EnvConfig.str("NBA_DEFINITION_ID", "def_union_needed")
SOURCE = EnvConfig.str("SOURCE", "calculator.union-needed.v1")
REQUEST_ID_PREFIX = EnvConfig.str("REQUEST_ID_PREFIX", "calc-union-needed")
DRY_RUN = EnvConfig.bool("DRY_RUN", False)  # Set false to POST events

In [3]:
def create_event_context(client, employee_count):
    context = {
        "client_id": client.get('client_id'),
        "employee_count": employee_count,
        "rule": f"employee_count > {EMPLOYEE_THRESHOLD}",
        "offer": "union_advisory",
        "message": "Your workforce size indicates union-related obligations may apply.",
        }
   
    return CalculatorUtils.set_context_hash(context)

def create_delete_event_context(client):
    context = {
        "client_id": client.get('client_id'),
        "rule": f"employee_count > {EMPLOYEE_THRESHOLD}",
        "offer": "union_advisory",
        "message": f"This NBA is no longer applicable to you, since employee_count < {EMPLOYEE_THRESHOLD}.",
        }
    context_hash = CalculatorUtils.calculate_hash(context)
    context["hash"] = context_hash

    return CalculatorUtils.set_context_hash(context)

In [4]:
new_events = []

active_events = [{'id': 'nba_b6fb222de4',
  'nba_definition_id': 'def_union_needed',
  'enterprise_number': '9876543210',
  'account_id': 'acc_456',
  'contact_id': None,
  'active': True,
  'status': 'new',
  'priority': 0,
  'context': {'client_id': 'client_003',
   'employee_count': 2,
   'rule': 'employee_count > 2',
   'offer': 'union_advisory',
   'message': 'Your workforce size indicates union-related obligations may apply.',
   'hash': '3dc508da42b75fe62966a738c5d0e3543efc4b8dddaeb374751d624260efac43'}},
 {'id': 'nba_cc73eabd14',
  'nba_definition_id': 'def_union_needed',
  'enterprise_number': '0123456789',
  'account_id': 'acc_123',
  'contact_id': None,
  'active': True,
  'status': 'new',
  'priority': 0,
  'context': {'client_id': 'client_001',
   'employee_count': 2,
   'rule': 'employee_count > 2',
   'offer': 'union_advisory',
   'message': 'Your workforce size indicates union-related obligations may apply.',
   'hash': '9341b6a6581c3673c1a0f255be1549ae7fc7d41daeea8b327280976991c6b707'}}]

In [ ]:
def find_inactive_events(active_events, new_events):
    active_event_hashes = {event.get('context', {}).get('hash',''): event for event in active_events}
    new_event_hashes = {event.get('context', {}).get('hash',''): event for event in new_events}

    hashes_to_remove = active_event_hashes.keys() - new_event_hashes.keys()
    return {k: active_event_hashes[k] for k in hashes_to_remove}

def create_deletion_events(events_to_remove):
    delete_events = []
    for event in events_to_remove.values():
        client = {"enterprise_number": event.get("enterprise_number"), "account_id": event.get("account_id"), "client_id": event.get("context").get("client_id")}
        
        delete_event_context = create_delete_event_context(client)
        delete_event = CalculatorUtils.create_delete_event(client, delete_event_context, [event.get('id')], nba_definition_id=NBA_DEFINITION_ID, source=SOURCE)
        
        delete_events.append(delete_event)

    return delete_events

In [6]:
inactive_events = find_inactive_events(active_events, new_events)
nbas_to_delete = create_deletion_events(inactive_events)

In [10]:
assert len(nbas_to_delete) == 2
assert nbas_to_delete[0]['context']['message'] == "This NBA is no longer applicable to you, since employee_count < 3."